In [78]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import activations
from tensorflow.keras import layers
import numpy as np 
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt 

In [79]:
#Residual blocks 
#Id function allows us to train a bigger network with the same layers.
    #=> We put X, keep X + a residual block learning the changes on X
    #=> Weights ---> 0 
    

In [80]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
print(x_train.shape)
print(y_train.shape)

LABELS = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
HEIGHT = x_train.shape[1]
WIDTH = x_train.shape[2]
INPUT_SHAPE = (HEIGHT, WIDTH, 3)

(50000, 32, 32, 3)
(50000, 1)


In [81]:
EPOCHS = 200
NUM_CLASSES = y_train.shape[0]

In [82]:
#Let's build the model ! 

#Resnet Block ! 
def residual_block(x, kernel, nb_filters, reduce=False): 
    result = x
    if reduce: 
        #Reduce shape => stride = 2  + 1x1
        #First conv if reduce 
        conv_result = tf.keras.layers.Conv2D(
            strides=2, 
            kernel_size=1,
            filters=nb_filters,
            padding="same", 
        )(result)
        #batch norm after each convolution 
        batch_norm_result = tf.keras.layers.BatchNormalization()(conv_result)
        result = tf.keras.layers.ReLU()(batch_norm_result)
    #Anyways run the first imposed conv
    conv_result = tf.keras.layers.Conv2D(
        strides=1, 
        kernel_size=kernel,
        filters=nb_filters,
        padding="SAME", 
    )(result)
    #Batch norm and activation (RELU)
    batch_norm_result = tf.keras.layers.BatchNormalization()(conv_result)
    result = tf.keras.layers.ReLU()(batch_norm_result)
    #Second imposed convolution 
    conv_result2 = tf.keras.layers.Conv2D(
        strides=1, 
        kernel_size=kernel, 
        filters=nb_filters, 
        padding="SAME", 
    )(result)
    #batch norm after each convolution 
    batch_norm_result2 = tf.keras.layers.BatchNormalization()(conv_result2)    
    activation_block_result = tf.keras.layers.ReLU()(batch_norm_result2)
    return activation_block_result     

In [87]:
#TODO 
# filters = [] => idx i = block i 
# nbr_filters = []
# res_block_nbr = [3, 4, 6, 3]
def ResNet(input_shape, filters, nbr_filters, res_block_nbr): 
    #start resnet HERE : 
        #7x7, 64 filters, stride = 2, padding = "same"
        #3x3 max pool, stride = 2
    
    #We assume we are in an 18 layer mindset => only 2 blocks of convolution by 
    inputs = tf.keras.Input(shape=input_shape)
    for step in range(len(filters)): 
        #Processing each convs
        #First Block = Reducing dimensions
        result = residual_block(
                    x=inputs, 
                    nb_filters=nbr_filters[step], 
                    kernel=filters[step], 
                    reduce=True
                )
        #Other blocks
        for i in range(1,res_block_nbr[step]): 
            result = residual_block(
                        x=result, 
                        nb_filters=nbr_filters[step], 
                        kernel=filters[step], 
                        reduce=False
                    )
    #Average pooling 
    result_final_pool = tf.keras.layers.AveragePooling2D()(result)
    #returining probabilities
    final_fc = tf.keras.layers.Dense(units=1000, activation="softmax")(result_final_pool)
    model = Model(inputs=inputs, outputs=final_fc)
    return model

In [89]:
model = ResNet(INPUT_SHAPE, filters=[3, 3, 3, 3], nbr_filters=[64,128,256,512], res_block_nbr=[3,4,6,3])

print(model.summary())

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_234 (Conv2D)          (None, 16, 16, 512)       2048      
_________________________________________________________________
batch_normalization_220 (Bat (None, 16, 16, 512)       2048      
_________________________________________________________________
re_lu_219 (ReLU)             (None, 16, 16, 512)       0         
_________________________________________________________________
conv2d_235 (Conv2D)          (None, 16, 16, 512)       2359808   
_________________________________________________________________
batch_normalization_221 (Bat (None, 16, 16, 512)       2048      
_________________________________________________________________
re_lu_220 (ReLU)             (None, 16, 16, 512)       0   